In [1]:
import nltk
import numpy
import tensorflow
import random
import json
import pickle
from nltk.stem.lancaster import LancasterStemmer

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
stemmer = LancasterStemmer()

In [5]:
with open("intents.json") as file:
  data = json.load(file)

In [6]:
print(data["intents"])

[{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'age', 'patterns': ['how old', 'how old is tim', 'what is your age', 'how old are you', 'age?'], 'responses': ['I am 20 years old!', '20 years young!'], 'context_set': ''}, {'tag': 'name', 'patterns': ['what is your name', 'what should I call you', 'whats your name?'], 'responses': ['You can call me Jim.', "I'm Jim!", "I'm Jim aka A.I."], 'context_set': ''}, {'tag': 'shop', 'patterns': ['Id like to buy something', 'whats on the menu', 'what do you reccommend?', 'could i get something to eat'], 'responses': ['We sell chocolate chip cookies for $2!', 'Cookies are on the menu!'], 'con

In [7]:
try:
  with open("data.pickle", "rb") as f:
    words, labels, training, output = pickle.load(f)
except:
  words = []
  labels = []
  docs_x = []
  docs_y = []

  for intent in data["intents"]:
   for pattern in intent["patterns"]:
     wrds = nltk.word_tokenize(pattern)
     words.extend(wrds)
     docs_x.append(wrds)
     docs_y.append(intent["tag"])

   if intent["tag"] not in labels:
     labels.append(intent["tag"])

  words = [stemmer.stem(w.lower())for w in words if w != "?" ]
  words = sorted(list(set(words)))
  labels = sorted(labels) 

  training = []
  output = []
  
  out_empty = [0 for _ in range(len(labels))]

  for x,doc in enumerate(docs_x):
    bag = []
    wrds = [stemmer.stem(w) for w in doc]

    for w in words:
      if w in wrds:
        bag.append(1)
      else:
        bag.append(0)
  
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

  training = numpy.array(training)
  output = numpy.array(output)

  with open("data.pickle","wb") as f:
    pickle.dump((words, labels, training, output), f)


In [8]:
# Example of how training will look
training

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [11]:
tensorflow.compat.v1.reset_default_graph()

In [14]:
# forgot to import above so import here
import tflearn

Instructions for updating:
non-resource variables are not supported in the long term


In [15]:
# Map of neural network we are using
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation = "softmax")
net = tflearn.regression(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [16]:
# Shape of output
output.shape

(26, 6)

In [17]:
model = tflearn.DNN(net)
model.fit(training, output, n_epoch= 1000, batch_size=8 , show_metric = True)
try:
  model.save("model.tflearn")
except:
  model.fit(training, output, n_epoch= 1000, batch_size =8, show_metric = True)

Training Step: 3999  | total loss: 0.89085 | time: 0.010s
| Adam | epoch: 1000 | loss: 0.89085 - acc: 0.9029 -- iter: 24/26
Training Step: 4000  | total loss: 0.73629 | time: 0.013s
| Adam | epoch: 1000 | loss: 0.73629 - acc: 0.9126 -- iter: 26/26
--


In [18]:
def bag_of_words(s, wordS):
  bag = [0 for _ in range(len(words))]

  s_words = nltk.word_tokenize(s)
  s_words = [stemmer.stem(word.lower()) for word in s_words]

  for se in s_words:
    for i, w in enumerate(words):
      if w == se:
        bag[i] = 1
  return numpy.array(bag)


In [26]:
def chat():
  print("Start talking with the bot (type quit to stop)!")
  while True:
    inp = input("You: ")
    if inp.lower() == "quit":
      break
    results =  model.predict([bag_of_words(inp,words)])[0]
    results_index = numpy.argmax(results)
    tag = labels[results_index]
    if results[results_index] >= 0.7 :
      for tg in data["intents"]:        
        if tg['tag'] == tag:
          responses = tg['responses']  
      print(random.choice(responses))
    else:
      print("I didn't get that, try again.")
chat()


Start talking with the bot (type quit to stop)!
You: what is your age
20 years young!
You: hi
Good to see you again!
You: quit
